In [1]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
100%|██████████████████████████████████████| 80.9M/80.9M [00:13<00:00, 6.93MB/s]
100%|██████████████████████████████████████| 80.9M/80.9M [00:13<00:00, 6.18MB/s]


In [2]:
!unzip sentiment140.zip

Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [15]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
columns = ['target', 'id', 'date', 'flag', 'user', 'text']
data = pd.read_csv('training.1600000.processed.noemoticon.csv', names = columns, encoding = 'ISO-8859-1')

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

#### Data Pre - Processing

In [ ]:
data.isnull().sum()

In [ ]:
data['target'].value_counts()

In [ ]:
# converting target value of 4 to 1
data.replace({'target': {4:1}}, inplace = True)

In [ ]:
data['target'].value_counts()

##### Stemming

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('^a-zA-Z', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english') ]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

In [ ]:
data['stemmed_content'] = data['text'].apply(stemming) 

In [ ]:
data.head()

In [ ]:
X = data['stemmed_content'].values
Y = data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state = 1)

In [ ]:
print(X_train)

In [ ]:
# converting textual data to numeric data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train[5])

#### Sentiment Analysis Using Logistic Regression

In [ ]:
clf = LogisticRegression()

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
y_train_pred = clf.predict(X_train)
print('Accuracy score on training data:', accuracy_score(Y_train, y_train_pred))

In [ ]:
y_test_pred = clf.predict(X_test)
print('Accuracy score on testing data:', accuracy_score(Y_test, y_test_pred))

#### Saving the trained model

In [ ]:
import pickle

filename = 'trained_model.sav'
pickle.dump(clf, open(filename, 'wb'))

#### Using the pre-trained model

In [ ]:
pre_trained = pickle.load(open('trained_model.sav', 'rb'))

In [ ]:
x_test = X_test[5]
y_test = Y_test[5]

In [ ]:
print(y_test)

In [ ]:
pred = pre_trained.predict(x_test)
if(pred[0] == 0):
    print("Negative Tweet")
else:
    print("Positive Tweet")

#### Sentiment Analysis Using Logistic Regression Using Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, Y_train)

In [ ]:
y_pred = nb.predict(X_test)

In [ ]:
print('Accuracy score on testing data:', accuracy_score(Y_test, y_pred))